In [1]:
%matplotlib inline 
from matplotlib import pyplot as plt

from mxnet import nd, autograd
from mxnet import gluon, init


In [26]:
n_train = 100
n_test = 100
w_true = [1.2, -3.4, 5.6] 
b_true = 5

# y = 1.2x - 3.4x^2 + 5.6x^3 + 5 + epsilon
features = nd.random.normal(0, 1, (n_train + n_test, 1))
poly_features = nd.concat(
    features, 
    nd.power(features, 2), 
    nd.power(features, 3)
)
labels = w_true[0] * poly_features[:, 0] \
    + w_true[1] * poly_features[:, 1] \
    + w_true[2] * poly_features[:, 2] \
    + b_true
labels += nd.random.normal(0, 0.01, labels.shape)


